In [157]:
# import api for threading
import concurrent.futures

# define sorting function
def mysort(arr):
    return sorted(arr)

# define MultiThreadSort function,that divide list into 3 equal parts and sort each subpart
# using different thread and finally merge those sorted subpart by another thread

def MultiThreadSort(arr):
    
    # check whether array can be divided into three equal part or not.
    if len(arr)%3==0:
        x = int(len(arr)/3)
        arr_thread1 = arr[:x]
        arr_thread2 = arr[x:2*x]
        arr_thread3 = arr[2*x:]
    else:
        print('Array can not be divided into 3 equal size,Please try different input')
        return
    
    # create 1st thread that execute mysort function with arr_thread1 (1st subpart) as array.
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread1 = executor.submit(mysort,arr_thread1)
        return_value1 = thread1.result()
        print('First Thread result: ',return_value1)

    # create 2nd thread that execute mysort function with arr_thread2 (2nd subpart) as array.
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread2 = executor.submit(mysort,arr_thread2)
        return_value2 = thread2.result()
        print('Second Thread result: ',return_value2)

    # create 3rd thread that execute mysort function with arr_thread3 (3rd subpart) as array.
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread3 = executor.submit(mysort,arr_thread3)
        return_value3 = thread3.result()
        print('Third thread result:  ',return_value3)

    # create a large array using results obtained from above three threads
    
    newarr = return_value1 + return_value2 + return_value3
    
    # Now create another thread to merge and sort.
    # it can be done using sorted(arr1,arr2,...)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        MergeThread = executor.submit(mysort,newarr)
        return_newarr = MergeThread.result()
        print('4th Thread, Merged result ',return_newarr)

In [158]:
arr = [5,2,4,8,9,22,11,54,0,6,3,14]

In [159]:
MultiThreadSort(arr)

First Thread result:  [2, 4, 5, 8]
Second Thread result:  [9, 11, 22, 54]
Third thread result:   [0, 3, 6, 14]
4th Thread, Merged result  [0, 2, 3, 4, 5, 6, 8, 9, 11, 14, 22, 54]


In [160]:
arr = [5,2,4,8,9,22,11]

In [161]:
MultiThreadSort(arr)

Array can not be divided into 3 equal size,Please try different input


In [162]:
import time, random
from threading import Thread
from sharedcell import SharedCell
from counter import Counter
from sharedcell import __init__

class Writer(Thread):
"""Increments a counter in a shared cell."""

    def __init__(self, cell, number):
    Thread.__init__(self, name = "Writer" + str(number))
    self.cell = cell

    def run(self):
    """Sleep for a random interval and then increment
    the counter in the cell."""
    print("%s starting up" % self.getName())
    time.sleep(random.randint(1, 4))
    # write, using counter's increment
    #value = ?
    print("%s is done incrementing to %d" % \
    (self.getName(), value))
    print()

class Reader(Thread):
"""Reads the value of a counter in a shared cell."""

    def __init__(self, cell, number):
    Thread.__init__(self, name = "Reader" + str(number))
    self.cell = cell

    def run(self):
    """Sleep for a random interval and then get
    the value of the counter in the cell."""
    print("%s starting up" % self.getName())
    time.sleep(random.randint(1, 4))
    # read, using counter's getValue
    #value = ?
    print("%s is done getting %d" % (self.getName(), value))
    print()

def main():
    """Creates a shared cell on a Counter object, and reader and
    writer threads to increment it and observe its value."""
    counter = Counter()
    cell = SharedCell(counter)
    threads = []
    print("Creating reader threads.")
    print()
    for i in range(1, 5):
    threads.append(Reader(cell, i))
    print("Creating writer threads.")
    for i in range(1, 3):
    threads.append(Writer(cell, i))
    print()
    print("Starting the threads.")
    print()
    for thread in threads:
    thread.start()

if __name__ == "__main__":
    main()

    """
    File: sharedcell.py
    Resource for shared data synchonization for the readers and writers
    problem. Guarantees that a writer finishes writing before readers can
    read and other writers can write. Also supports concurrent reading.
    """

    from threading import Condition

    class SharedCell(object):
        """Synchronizes readers and writers around shared data,
        to support thread-safe reading and writing."""

        def __init__(self, data):
        """Sets up the conditions and count of active readers."""
        self.data = data
        self.writing = False
        self.readerCount = 0
        self.okToRead = Condition()
        self.okToWrite = Condition()

        def beginRead(self):
        """Waits until a writer is not writing or the writers
        condition queue is empty. Then increments the reader
        count and notifies the next waiting reader."""
        self.okToRead.acquire()
        self.okToWrite.acquire()
        while self.writing or len(self.okToWrite._waiters) > 0:
        self.okToRead.wait()
        self.readerCount += 1
        self.okToRead.notify()

        def endRead(self):
        """Notifies a waiting writer if there are
        no active readers."""
        self.readerCount -= 1
        if self.readerCount == 0:
        self.okToWrite.notify()
        self.okToWrite.release()
        self.okToRead.release()

        def beginWrite(self):
        """Can write only when someone else is not
        writing and there are no readers are ready."""
        self.okToWrite.acquire()
        self.okToRead.acquire()
        while self.writing or self.readerCount != 0:
        self.okToWrite.wait()
        self.writing = True

        def endWrite(self):
            """Notify the next waiting writer if the readers
            condition queue is empty. Otherwise, notify the
            next waiting reader."""
            self.writing = False
            if len(self.okToRead._waiters) > 0:
            self.okToRead.notify()
            else:
            self.okToWrite.notify()
            self.okToRead.release()
            self.okToWrite.release()

        def read(self, readerFunction):
            """Observe the data in the shared cell."""
            self.beginRead()
            # Enter reader's critical section
            result = readerFunction(self.data)
            # Exit reader's critical section
            self.endRead()
            return result

        def write(self, writerFunction):
            """Modify the data in the shared cell."""
            self.beginWrite()
            # Enter writer's critical section
            result = writerFunction(self.data)
            # Exit writer's critical section
            self.endWrite()
            return result

"""
File: counter.py
"""

class Counter(object):
    """Models a counter."""

    # Constructor
    def __init__(self):
    """Sets up the counter."""
    self.reset()

    # Mutator methods
    def reset(self):
    """Returns the counter to 0."""
    self.value = 0

    def increment(self, amount = 1):
    """Adds amount to the counter."""
    self.value += amount
    return self.value

    def decrement(self, amount = 1):
    """Subtracts amount from the counter."""
    self.value -= amount
    #return self.value

    # Accessor methods
    def getValue(self):
    """Returns the counter's value."""
    return self.value

    def __str__(self):
    """Returns the string representation of the counter."""
    return str(self.value)

    def __eq__(self, other):
    """Returns True if self equals other or False otherwise."""
    if type(self) is type(other):
    return True
    if type(self) != type(other):
    return False
    return self.value == other._value

IndentationError: expected an indented block (<ipython-input-162-cd49dbcbbc5a>, line 8)